In [1]:
import os
import numpy as np

In [ ]:
# # Download YOLOv7 repository and install requirements
# !git clone https://github.com/WongKinYiu/yolov7
# %cd yolov7
# %pip install -r requirements.txt

In [ ]:
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

# Yolo augment data internally so no need to augment it

In [1]:
!python ./train.py --batch 4 --epochs 2 --data ./data/data.yaml --weights './runs/train/yolov73/weights/best.pt' --name yolov7 --img 640 640 --cf cfg/training/yolov7.yaml --hyp ./data/hyp.scratch.p5.yaml --notest --noautoanchor --freeze 2
## Training with diffrent hyperparameters
# !python ./train.py --batch 8 --epochs 4 --data ./data/data.yaml --weights './runs/train/yolov7/weights/best.pt' --name yolov7 --img 640 640 --cf cfg/training/yolov7.yaml --hyp ./data/hyp.scratch.p6.yaml --notest --noautoanchor --freeze 2


2024-08-14 21:38:45.923783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 21:38:46.129959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 21:38:46.194605: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 21:38:46.552180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 21:38:49.794273: W tensorflow/compiler/tf2

In [3]:
!python ./test.py --conf-thres 0.2 --batch-size 4 --data ./data/data.yaml --weights './runs/train/yolov73/weights/best.pt' --name yolov7 --img-size 640

Namespace(weights=['./runs/train/yolov73/weights/best.pt'], data='./data/data.yaml', batch_size=4, img_size=640, conf_thres=0.2, iou_thres=0.65, task='val', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolov7', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 df6b987 torch 2.3.1+cu121 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/home/charbel/.local/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36524924 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is trace

# Evaluation

We can evaluate the performance of our custom training using the provided evalution script.

Note we can adjust the below custom arguments. For details, see [the arguments accepted by detect.py](https://github.com/WongKinYiu/yolov7/blob/main/detect.py#L154).


In [ ]:
# Run evaluation
!python ./detect.py --weights ./runs/train/yolov73/weights/best.pt --conf 0.1 --source ../yolov7_data/test/images


In [ ]:
!python yolov7/export.py --weights ./yolov7/runs/train/yolov74/weights/best.pt --img-size 1280 736 --batch-size 1 --include-nms

In [ ]:
# display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000  # max images to print
for imageName in glob.glob("./runs/detect/exp/*.png"):  # assuming JPG
    if i < limit:
        display(Image(filename=imageName))
        print("\n")
    i = i + 1

# Reparameterize for Inference

https://github.com/WongKinYiu/yolov7/blob/main/tools/reparameterization.ipynb


# OPTIONAL: Deployment

To deploy, you'll need to export your weights and save them to use later.


In [ ]:
# optional, zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/yolov73/best.pt
!zip export.zip runs/train/exp/*

# OPTIONAL: Active Learning Example

Once our first training run is complete, we should use our model to help identify which images are most problematic in order to investigate, annotate, and improve our dataset (and, therefore, model).

To do that, we can execute code that automatically uploads images back to our hosted dataset if the image is a specific class or below a given confidence threshold.


In [ ]:
# # setup access to your workspace
# rf = Roboflow(api_key="YOUR_API_KEY")                               # used above to load data
# inference_project =  rf.workspace().project("YOUR_PROJECT_NAME")    # used above to load data
# model = inference_project.version(1).model

# upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# print("inference reference point: ", inference_project)
# print("upload destination: ", upload_project)

In [ ]:
# # example upload: if prediction is below a given confidence threshold, upload it

# confidence_interval = [10,70]                                   # [lower_bound_percent, upper_bound_percent]

# for prediction in predictions:                                  # predictions list to loop through
#   if(prediction['confidence'] * 100 >= confidence_interval[0] and
#           prediction['confidence'] * 100 <= confidence_interval[1]):

#           # upload on success!
#           print(' >> image uploaded!')
#           upload_project.upload(image, num_retry_uploads=3)     # upload image in question

# Next steps

Congratulations, you've trained a custom YOLOv7 model! Next, start thinking about deploying and [building an MLOps pipeline](https://docs.roboflow.com) so your model gets better the more data it sees in the wild.


In [2]:
!tensorboard --logdir ./runs/train/yolov73

2024-08-15 10:33:09.907920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 10:33:09.965049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 10:33:09.982709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 10:33:10.023101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 10:33:12.844081: W tensorflow/compiler/tf2